In [2]:
import cv2
import numpy as np
import requests
from tensorflow.keras.models import load_model
import time
from tensorflow.keras.preprocessing import image

In [ ]:
# Raspberry Pi's IP address
raspberry_pi_ip = 'http://10.10.10.10:5001/video'

#Load the model
model = load_model("model_v1test.keras")

# Open the video strem
cap = cv2.VideoCapture(raspberry_pi_ip)

# Server endpoint to send the message back
host_url = 'http://10.10.10.10:5001/update'  # Modify this to match your server endpoint

In [ ]:
a=0
while a<5:
    # ready = input('Ready? ')
    ret, frame = cap.read()
    if not ret:
        break
    
    cv2.imshow("Live feed", frame)

    # Resize the frame to 150x150 
    frame = cv2.resize(frame, (150, 150))

    # Normalize the frame for model input
    img_array = (frame / 127.5) - 1.0  

    # Expand dimensions to match model' input shape 
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    print(f"Predictions: {prediction}")

    #Find the classification with the highest percentage of being true
    message=None
    max=0
    for count, prob in enumerate(prediction[0]):
        if prob > max:
            max = prob
            message = str(count)
    print(f"Final prediction: {message}")

    # Send the message back to the host if a valid message is detected
    if message is not None:
        response = requests.post(host_url, json={'message': message})
        if response.status_code == 200:
            print(f"Message sent to host: {message}")
        else:
            print(f"Failed to send message. Status code: {response.status_code}")
    time.sleep(5)
    a+=1

    # Exit if q pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predictions: [[1.8363536e-04 9.9540579e-01 1.8600130e-03 2.5505887e-03]]
Final prediction: 1
Message sent to host: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predictions: [[1.8326161e-04 9.9541312e-01 1.8570259e-03 2.5466587e-03]]
Final prediction: 1
Message sent to host: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predictions: [[1.8294145e-04 9.9541926e-01 1.8544615e-03 2.5432853e-03]]
Final prediction: 1
Message sent to host: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predictions: [[1.8269494e-04 9.9542409e-01 1.8524741e-03 2.5406624e-03]]
Final prediction: 1
Message sent to host: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predictions: [[1.8246280e-04 9.9542868e-01 1.8506037e-03 2.5382086e-03]]
Final prediction: 1
Message sent to host: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predictions: [[1.8220673e-04 9.9543375e-01 1.8485441e-03 2.5354975e-03]]
Final prediction: 1
Message sent to host: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predictions: [[1.8197365e-04 9.954